In [53]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [54]:
node = "lotus"

In [55]:
#import exp_filtered
data = pd.read_csv('../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
409,891,2024-02-15 19:25:21.9807959,2024-02-15 19:55:21.9832958,validator,random33,19,8,23,4,8,2,0.25,3.00,1708025121,1708026921
412,898,2024-02-15 23:20:24.1114255,2024-02-15 23:50:24.1747705,validator,random5,8,6,16,4,8,2,0.25,0.25,1708039224,1708041024
413,899,2024-02-15 23:53:58.9555104,2024-02-16 00:23:59.0175630,validator,random5,8,6,16,4,8,2,0.25,0.25,1708041238,1708043039
415,901,2024-02-16 01:01:07.9360686,2024-02-16 01:31:07.9411590,validator,random6,16,8,20,6,8,2,0.25,1.00,1708045267,1708047067
416,902,2024-02-16 01:34:42.7924620,2024-02-16 02:04:42.7974999,validator,random6,16,8,20,6,8,2,0.25,1.00,1708047282,1708049082
418,904,2024-02-16 02:41:52.3057095,2024-02-16 03:11:52.3106811,validator,random7,10,8,16,4,8,4,0.25,1.00,1708051312,1708053112
419,905,2024-02-16 03:15:26.8680015,2024-02-16 03:45:26.8705969,validator,random7,10,8,16,4,8,4,0.25,1.00,1708053326,1708055126
421,907,2024-02-16 04:22:35.4565262,2024-02-16 04:52:35.4619071,validator,random8,10,8,16,4,8,2,0.15,1.00,1708057355,1708059155
422,908,2024-02-16 04:56:10.5331406,2024-02-16 05:26:10.5389618,validator,random8,10,8,16,4,8,2,0.15,1.00,1708059370,1708061170
423,909,2024-02-16 05:29:45.3956405,2024-02-16 05:59:45.3984223,validator,reference,8,6,12,4,8,2,0.25,1.00,1708061385,1708063185


In [56]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/mess_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,experiment,type
0,2024-02-16 05:38:10.128407+00:00,909,16.0
0,2024-02-16 05:36:59.010232+00:00,909,17.0
1,2024-02-16 05:36:59.218708+00:00,909,17.0
2,2024-02-16 05:37:00.010101+00:00,909,17.0
3,2024-02-16 05:37:00.216797+00:00,909,17.0
4,2024-02-16 05:37:00.974969+00:00,909,17.0
5,2024-02-16 05:37:00.981660+00:00,909,17.0
6,2024-02-16 05:37:00.983129+00:00,909,17.0
7,2024-02-16 05:37:00.987263+00:00,909,17.0
8,2024-02-16 05:37:01.010522+00:00,909,17.0


In [57]:
# Transpose the table -> type and count
df = messages#.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='5s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="5s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_82388/2738252698.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:00+00:00,345.0,56.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,466.0,73.0,0.0,0.0,0.0,64.0
2,390,2023-10-04 18:52:10+00:00,781.0,124.0,0.0,0.0,0.0,64.0
3,390,2023-10-04 18:52:15+00:00,734.0,111.0,0.0,0.0,0.0,32.0
4,390,2023-10-04 18:52:20+00:00,554.0,91.0,0.0,0.0,0.0,64.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,776.0,125.0,0.0,0.0,0.0,64.0
96,390,2023-10-04 19:00:00+00:00,479.0,74.0,0.0,0.0,0.0,32.0
97,390,2023-10-04 19:00:05+00:00,820.0,129.0,0.0,0.0,0.0,64.0
98,390,2023-10-04 19:00:10+00:00,821.0,134.0,0.0,0.0,0.0,64.0


In [58]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

# eventsTS.to_csv('./events20s.csv')
# eventsTS.head(500)

In [59]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

query_mess = pd.DataFrame()
query_pub = pd.DataFrame()

for index, row in experiments.iterrows():
    if os.path.exists('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_mess = pd.DataFrame(data)
        deliverMessage = pd.concat([query_mess, deliverMessage])
        del query_mess

    if os.path.exists('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_pub = pd.DataFrame(data)
        publishMessage = pd.concat([publishMessage, query_pub])
        del query_pub

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-02-16 05:36:58.680059+00:00,ACQIARIgutFeCGDvY8CEGLKJLbwhzAojBcANvaPtrqW25p...,909
1,2024-02-16 05:36:58.766329+00:00,ACQIARIg9J81Yj/smCNF6/9o0XZmWCIQLH5scTrDUQ/5Fy...,909
2,2024-02-16 05:37:00.095781+00:00,ACQIARIg9J81Yj/smCNF6/9o0XZmWCIQLH5scTrDUQ/5Fy...,909
3,2024-02-16 05:37:00.802130+00:00,ACQIARIgZTwM9zqvuGbntFGs3WCNz/6u9RqIjPBSg9JdSf...,909
4,2024-02-16 05:37:00.858310+00:00,ACQIARIgfl7Wkeq304/jtKRO9+BFSqGblHME8UBHqKYV1B...,909
5,2024-02-16 05:37:01.005434+00:00,ACQIARIgDyYdTccrMyzaXMarwZL3ceYmaR0Y/e7nmYKRbb...,909
6,2024-02-16 05:37:01.019659+00:00,ACQIARIg2E5YMFznHacBOytrj+HcCNQBtpgMqFDEHtnRlQ...,909
7,2024-02-16 05:37:03.099158+00:00,ACQIARIg9J81Yj/smCNF6/9o0XZmWCIQLH5scTrDUQ/5Fy...,909
8,2024-02-16 05:37:03.806896+00:00,ACQIARIgZTwM9zqvuGbntFGs3WCNz/6u9RqIjPBSg9JdSf...,909
9,2024-02-16 05:37:03.864446+00:00,ACQIARIgfl7Wkeq304/jtKRO9+BFSqGblHME8UBHqKYV1B...,909


In [60]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.978680+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.978989+00:00,309
1,2023-10-04 18:52:04.979101+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.979353+00:00,252
2,2023-10-04 18:52:04.979467+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.979741+00:00,274
3,2023-10-04 18:52:04.979869+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.980060+00:00,191
4,2023-10-04 18:52:04.980205+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.980706+00:00,501
5,2023-10-04 18:52:04.980533+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:04.980758+00:00,225
6,2023-10-04 18:52:07.987652+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.988025+00:00,373
7,2023-10-04 18:52:07.988149+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.988422+00:00,273
8,2023-10-04 18:52:07.988543+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.989014+00:00,471
9,2023-10-04 18:52:07.988927+00:00,ACQIARIgy9UsLGQXZgxfmle06NvWnMG6X8Lgry2zmzr66e...,390,2023-10-04 18:52:07.989328+00:00,401


In [61]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='5s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_82388/2407133802.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:00+00:00,292.000000
1,390,2023-10-04 18:52:05+00:00,373.333333
2,390,2023-10-04 18:52:10+00:00,649.083333
3,390,2023-10-04 18:52:15+00:00,337.666667
4,390,2023-10-04 18:52:20+00:00,234.000000
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,391.166667
96,390,2023-10-04 19:00:00+00:00,430.333333
97,390,2023-10-04 19:00:05+00:00,404.166667
98,390,2023-10-04 19:00:10+00:00,690.333333


In [62]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,391.166667,776.0,125.0,0.0,0.0,0.0,64.0
96,390,2023-10-04 19:00:00+00:00,430.333333,479.0,74.0,0.0,0.0,0.0,32.0
97,390,2023-10-04 19:00:05+00:00,404.166667,820.0,129.0,0.0,0.0,0.0,64.0
98,390,2023-10-04 19:00:10+00:00,690.333333,821.0,134.0,0.0,0.0,0.0,64.0


In [63]:
rpc = pd.DataFrame()

for idx, exec in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/rpc_'+node+'_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.135930+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135962+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.139095+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.139121+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.139132+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.139214+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.139248+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.139264+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.139276+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.139293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [64]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='5s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_82388/1580870231.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [65]:
message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='5s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_82388/1432395461.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:00+00:00,56
1,390,2023-10-04 18:52:05+00:00,73
2,390,2023-10-04 18:52:10+00:00,124
3,390,2023-10-04 18:52:15+00:00,111
4,390,2023-10-04 18:52:20+00:00,91
5,390,2023-10-04 18:52:25+00:00,124
6,390,2023-10-04 18:52:30+00:00,105
7,390,2023-10-04 18:52:35+00:00,113
8,390,2023-10-04 18:52:40+00:00,127
9,390,2023-10-04 18:52:45+00:00,91


In [66]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='5s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_82388/448925867.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,452
1,390,2023-10-04 18:52:05+00:00,638
2,390,2023-10-04 18:52:10+00:00,1050
3,390,2023-10-04 18:52:15+00:00,967
4,390,2023-10-04 18:52:20+00:00,757
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,1046
96,390,2023-10-04 19:00:00+00:00,637
97,390,2023-10-04 19:00:05+00:00,1098
98,390,2023-10-04 19:00:10+00:00,1106


In [67]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,452
1,390,2023-10-04 18:52:05+00:00,638
2,390,2023-10-04 18:52:10+00:00,1050
3,390,2023-10-04 18:52:15+00:00,967
4,390,2023-10-04 18:52:20+00:00,757
5,390,2023-10-04 18:52:25+00:00,1018
6,390,2023-10-04 18:52:30+00:00,892
7,390,2023-10-04 18:52:35+00:00,893
8,390,2023-10-04 18:52:40+00:00,1066
9,390,2023-10-04 18:52:45+00:00,791


In [68]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipReceived'] = gossipBandwidth['count']
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/5
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,396,79.2
1,390,2023-10-04 18:52:05+00:00,565,113.0
2,390,2023-10-04 18:52:10+00:00,926,185.2
3,390,2023-10-04 18:52:15+00:00,856,171.2
4,390,2023-10-04 18:52:20+00:00,666,133.2
...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,921,184.2
96,390,2023-10-04 19:00:00+00:00,563,112.6
97,390,2023-10-04 19:00:05+00:00,969,193.8
98,390,2023-10-04 19:00:10+00:00,972,194.4


In [69]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/5
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,11.2
1,390,2023-10-04 18:52:05+00:00,14.6
2,390,2023-10-04 18:52:10+00:00,24.8
3,390,2023-10-04 18:52:15+00:00,22.2
4,390,2023-10-04 18:52:20+00:00,18.2
5,390,2023-10-04 18:52:25+00:00,24.8
6,390,2023-10-04 18:52:30+00:00,21.0
7,390,2023-10-04 18:52:35+00:00,22.6
8,390,2023-10-04 18:52:40+00:00,25.4
9,390,2023-10-04 18:52:45+00:00,18.2


In [70]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_82388/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2
...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:27:00+00:00,400.083333,1376.0,128.0,0.0,0.0,0.0,10.0,25.6
496,394,2023-10-04 21:27:05+00:00,559.583333,1274.0,118.0,0.0,0.0,0.0,10.0,23.6
497,394,2023-10-04 21:27:10+00:00,379.500000,891.0,82.0,0.0,0.0,0.0,10.0,16.4
498,394,2023-10-04 21:27:15+00:00,477.000000,1321.0,122.0,0.0,0.0,0.0,10.0,24.4


In [71]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/5

totalBand.head(500)

/tmp/ipykernel_82388/511213079.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/511213079.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2
...,...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:27:00+00:00,400.083333,1376.0,128.0,0.0,0.0,0.0,10.0,25.6,600.8
496,394,2023-10-04 21:27:05+00:00,559.583333,1274.0,118.0,0.0,0.0,0.0,10.0,23.6,555.4
497,394,2023-10-04 21:27:10+00:00,379.500000,891.0,82.0,0.0,0.0,0.0,10.0,16.4,389.0
498,394,2023-10-04 21:27:15+00:00,477.000000,1321.0,122.0,0.0,0.0,0.0,10.0,24.4,575.8


In [72]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_82388/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4,396.0,79.2
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8,565.0,113.0
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2,926.0,185.2
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2,856.0,171.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2,666.0,133.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,394,2023-10-04 21:27:00+00:00,400.083333,1376.0,128.0,0.0,0.0,0.0,10.0,25.6,600.8,1500.0,300.0
496,394,2023-10-04 21:27:05+00:00,559.583333,1274.0,118.0,0.0,0.0,0.0,10.0,23.6,555.4,1385.0,277.0
497,394,2023-10-04 21:27:10+00:00,379.500000,891.0,82.0,0.0,0.0,0.0,10.0,16.4,389.0,972.0,194.4
498,394,2023-10-04 21:27:15+00:00,477.000000,1321.0,122.0,0.0,0.0,0.0,10.0,24.4,575.8,1436.0,287.2


In [73]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4,396.0,79.2
1,390,2023-10-04 18:52:05+00:00,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8,565.0,113.0
2,390,2023-10-04 18:52:10+00:00,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2,926.0,185.2
3,390,2023-10-04 18:52:15+00:00,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2,856.0,171.2
4,390,2023-10-04 18:52:20+00:00,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2,666.0,133.2
5,390,2023-10-04 18:52:25+00:00,381.500000,750.0,124.0,0.0,0.0,0.0,64.0,24.8,353.6,894.0,178.8
6,390,2023-10-04 18:52:30+00:00,328.166667,671.0,105.0,0.0,0.0,0.0,32.0,21.0,312.6,787.0,157.4
7,390,2023-10-04 18:52:35+00:00,349.722222,653.0,113.0,0.0,0.0,0.0,64.0,22.6,309.2,780.0,156.0
8,390,2023-10-04 18:52:40+00:00,581.083333,792.0,127.0,0.0,0.0,0.0,64.0,25.4,371.6,939.0,187.8
9,390,2023-10-04 18:52:45+00:00,473.166667,598.0,91.0,0.0,0.0,0.0,32.0,18.2,277.8,700.0,140.0


In [74]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../Datasets/consolidated5s_'+node+'.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,...,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,292.000000,345.0,56.0,0.0,0.0,0.0,0.0,11.2,159.4,...,79.2,unl,8,6,12,4,8,2,0.25,1.0
1,390,373.333333,466.0,73.0,0.0,0.0,0.0,64.0,14.6,220.8,...,113.0,unl,8,6,12,4,8,2,0.25,1.0
2,390,649.083333,781.0,124.0,0.0,0.0,0.0,64.0,24.8,366.2,...,185.2,unl,8,6,12,4,8,2,0.25,1.0
3,390,337.666667,734.0,111.0,0.0,0.0,0.0,32.0,22.2,340.2,...,171.2,unl,8,6,12,4,8,2,0.25,1.0
4,390,234.000000,554.0,91.0,0.0,0.0,0.0,64.0,18.2,262.2,...,133.2,unl,8,6,12,4,8,2,0.25,1.0
5,390,381.500000,750.0,124.0,0.0,0.0,0.0,64.0,24.8,353.6,...,178.8,unl,8,6,12,4,8,2,0.25,1.0
6,390,328.166667,671.0,105.0,0.0,0.0,0.0,32.0,21.0,312.6,...,157.4,unl,8,6,12,4,8,2,0.25,1.0
7,390,349.722222,653.0,113.0,0.0,0.0,0.0,64.0,22.6,309.2,...,156.0,unl,8,6,12,4,8,2,0.25,1.0
8,390,581.083333,792.0,127.0,0.0,0.0,0.0,64.0,25.4,371.6,...,187.8,unl,8,6,12,4,8,2,0.25,1.0
9,390,473.166667,598.0,91.0,0.0,0.0,0.0,32.0,18.2,277.8,...,140.0,unl,8,6,12,4,8,2,0.25,1.0
